In [1]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()

In [2]:
class Borehole(Base):
    """The Boreholes Info table"""
    __tablename__ = 'Boreholes'
    id = Column(String(32), primary_key=True)
    intervals = relationship(
        'Interval',
        collection_class=attribute_mapped_collection('id'),
        cascade='all, delete-orphan')
    intervals_values = association_proxy(
        'intervals', 'value',
        creator=lambda k, v: Interval(id=k, description=v))

class Interval(Base):
    """The Interval table"""
    __tablename__ = 'Intervals'
    borehole = Column(String(32), ForeignKey('Boreholes.id'),
                           primary_key=True)
    # Note that this could be a numeric ID as well
    id = Column(Integer, primary_key=True)
    components = relationship('Component',secondary='Linkintervalcomponent')
    description = Column(String(32))
    
class Component(Base):
    """The Component table"""
    __tablename__ = 'Components'
    id = Column(String(32), primary_key=True)
    intervals = relationship(Interval,secondary='Linkintervalcomponent')
    description = Column(String(32))

class LinkIntervalComponent(Base):
    __tablename__ = 'Linkintervalcomponent'
    bh_id = Column(
        Integer, 
        ForeignKey('Intervals.borehole'), 
        primary_key = True)

    int_id = Column(
        Integer, 
        ForeignKey('Intervals.id'), 
        primary_key = True)

    comp_id = Column(
       Integer, 
       ForeignKey('Components.id'), 
       primary_key = True)

In [3]:
engine = create_engine('sqlite:///test_db3.db', echo=True)

In [4]:
Base.metadata.create_all(engine)

2020-12-04 12:30:39,111 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-04 12:30:39,114 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 12:30:39,117 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-04 12:30:39,118 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 12:30:39,121 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-04 12:30:39,122 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 12:30:39,124 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-04 12:30:39,125 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 12:30:39,126 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Intervals")
2020-12-04 12:30:39,127 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 12:30:39,128 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Intervals")
2020-12-04 12:30:39,128 INFO sqlalchemy.engine.base.Engine ()
2020-12-04

In [5]:
from contextlib import contextmanager

Session = sessionmaker(bind=engine)

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [6]:
boreholes = [Borehole(id='F11'), Borehole(id='F12'), Borehole(id='F13')] 

AmbiguousForeignKeysError: Could not determine join condition between parent/child tables on relationship Interval.components - there are multiple foreign key paths linking the tables via secondary table 'Linkintervalcomponent'.  Specify the 'foreign_keys' argument, providing a list of those columns which should be counted as containing a foreign key reference from the secondary table to each of the parent and child tables.

In [7]:
boreholes[0].intervals_values = {0: 'stuff', 1: 'junk'}
boreholes[1].intervals_values = {0: 'foo', 1: 'bar'}

In [15]:
c1 = Component(**{'id':0, 'description':'limestone'})
c1.intervals.append(0)

In [8]:
with session_scope() as session:
    for bh in boreholes:
        session.add(bh)

2020-12-04 12:24:21,942 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-04 12:24:21,944 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-12-04 12:24:21,945 INFO sqlalchemy.engine.base.Engine (('F11',), ('F12',), ('F13',))
2020-12-04 12:24:21,950 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (borehole, id, description) VALUES (?, ?, ?)
2020-12-04 12:24:21,951 INFO sqlalchemy.engine.base.Engine (('F11', 0, 'stuff'), ('F11', 1, 'junk'), ('F12', 0, 'foo'), ('F12', 1, 'bar'))
2020-12-04 12:24:21,953 INFO sqlalchemy.engine.base.Engine COMMIT
